In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import camb 
from camb import model
print(f'Using CAMB {camb.__version__} installed at {os.path.dirname(camb.__file__)}')

Using CAMB 1.5.8 installed at /home/gilee/CAMB/camb


In [2]:
def initialize_camb_params(cosmo_params, z):
    
    h = cosmo_params['h']
    Omega_b = cosmo_params['Omega_b']
    Omega_c = cosmo_params['Omega_c']
    Omega_k = cosmo_params['Omega_k']
    w0 = cosmo_params['w0']
    wa = cosmo_params['wa']
    As = cosmo_params['As']
    ns = cosmo_params['ns']
    kmax = cosmo_params['kmax']
    k_per_logint = cosmo_params['k_per_logint']
    
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=100*h, ombh2=Omega_b*h*h, omch2=Omega_c*h*h, omk=Omega_k)
    pars.DarkEnergy = camb.dark_energy.DarkEnergyPPF(w=w0, wa=wa)
    pars.InitPower.set_params(As=As, ns=ns)
    pars.set_matter_power(redshifts=z, kmax=kmax, k_per_logint=k_per_logint)
    pars.NonLinear = model.NonLinear_none
    
    return pars

In [3]:
def adjust_As_at_z0(pars, cosmo_params):
    
    z0 = pars.Transfer.PK_redshifts[-1] # as CAMB sorts redshifts (earliest first)

    if z0 != 0:
        print(f"Redshifts should incldue 0 here, as we normalize sigma8 at z=0.")
        return None

    results = camb.get_results(pars)
    sigma8_calc = results.get_sigma8()[-1] # sigma8 at z=0
    sigma8_goal = cosmo_params['sigma8_z0_WMAP5'] # target sigma8 at z=0

    print(f"Target     sigma8 = {sigma8_goal} at z = {z0}")
    print(f"Initial    sigma8 = {sigma8_calc}, As = {pars.InitPower.As} (trial)")

    As = pars.InitPower.As * (sigma8_goal / sigma8_calc)**2 # new normalization
    pars.InitPower.set_params(As=As, ns=pars.InitPower.ns) # set new parameterization

    # Get new results and sigma8 value
    results = camb.get_results(pars)
    sigma8 = results.get_sigma8()[-1] # new sigma8 at z=0
    print(f"Normalized sigma8 = {sigma8}, diff={sigma8 - sigma8_goal}, As = {pars.InitPower.As} (new)")
    
    return pars

In [4]:
bias8_z0_WMAP5  = 1.26             # target value # bias8 at z=0

As0       = 2e-9                   # trial value # power spectrum noramlization factor at k=k0
ns        = 0.96                   # power spectral index
h         = 0.72                   # little h, dimensionless hubble parameter, H0 = 100*h [km/s/Mpc]

Omega_m   = 0.26                   # physical total matter density
Omega_b   = 0.044                  # physical baryon density
Omega_k   = 0.0                    # spatial curvature

# EoS of DE
w0 = -1
wa = 0

In [5]:
# For given parameters, Lbox will be determined.
z_start    = 200  # starting redshift
levelmin   = 10   # initial cell size = 2**levelmin
dx_fin_kpc = 1    # [kpc] or [ckpc]: final resolution at z=0
FlagAMR    = 1    # flag for AMR
FlagZoom   = 0    # flag for zoom-in IC

print("For given paramters:")
print(f"  - z_start   = {z_start}")
print(f"  - levelmin  = {levelmin} (#cell = {2**levelmin})")
print(f"  - dx_fin    = {dx_fin_kpc} [kpc] at z = 0 (target resolution)")
print(f"  - FlagAMR   = {FlagAMR} (1: Turn on AMR, 0: Turn off AMR)")
print(f"  - FlagZoom  = {FlagZoom} (1: Zoom-in, 0: uni-grid)")

For given paramters:
  - z_start   = 200
  - levelmin  = 10 (#cell = 1024)
  - dx_fin    = 1 [kpc] at z = 0 (target resolution)
  - FlagAMR   = 1 (1: Turn on AMR, 0: Turn off AMR)
  - FlagZoom  = 0 (1: Zoom-in, 0: uni-grid)


In [6]:
# Starting redshift and scale factor
a_start = 1/(z_start + 1)

# Global refinemnent times (fixed)
if FlagAMR == 0: a_refine = np.array([])
if FlagAMR == 1: a_refine = np.array([0.0125, 0.025, 0.05, 0.1, 0.2, 0.4, 0.8])
z_refine = 1/a_refine - 1

# Level settins
levelmax = levelmin + len(a_refine) # level at z=0
# If a_start < a_refine.min(), just add len(a_refine).
# If a_start > a_refine.min(), the code has to be modified.

# Box size calculation
n_cell0 = 2**levelmin          # initial number of cell
c_Lbox_kpc = 2**levelmax * dx_fin_kpc # [ckpc] comoving box size
c_Lbox_Mpc = c_Lbox_kpc/1000   # [cMpc] comoving box size

# Print the parameters
print(f"Starting redshift (z0): {z_start}")
print(f"Starting scale factor (a0): 1/{z_start} = {a_start}")
print(f"Global refinement times:")
for a_r, z_r in zip(a_refine, z_refine):
    print(f"  a = {a_r:10.4f}, z = {z_r:10.4f}")
print(f"Minimum level (levelmin): {levelmin}")
print(f"Maximum level (levelmax): {levelmax}")
print(f"Comoving box size (c_Lbox) in cMpc: {c_Lbox_Mpc:10.4f}")
print(f"Comoving box size (c_Lbox) in cMpc/h: {c_Lbox_Mpc*h:10.4f}")

Starting redshift (z0): 200
Starting scale factor (a0): 1/200 = 0.004975124378109453
Global refinement times:
  a =     0.0125, z =    79.0000
  a =     0.0250, z =    39.0000
  a =     0.0500, z =    19.0000
  a =     0.1000, z =     9.0000
  a =     0.2000, z =     4.0000
  a =     0.4000, z =     1.5000
  a =     0.8000, z =     0.2500
Minimum level (levelmin): 10
Maximum level (levelmax): 17
Comoving box size (c_Lbox) in cMpc:   131.0720
Comoving box size (c_Lbox) in cMpc/h:    94.3718


In [7]:
# power spectrum params
kmin = 2 * np.pi / c_Lbox_Mpc       # [1/Mpc] # largest scale (box size)
kmax = np.pi * n_cell0 / c_Lbox_Mpc # [1/Mpc] # smallest scale (Nyquist wavenumber)
khmin = kmin/h                      # [1/Mpc] # largest scale (box size)
khmax = kmax/h                      # [1/Mpc] # smallest scale (Nyquist wavenumber)
k0  = 0.05                          # [1/Mpc] pivot_scalar
kh0 = k0/h                          # [1/Mpc] pivot_scalar
k_per_logint = 0 # default: 0 # affects the number of k values

print("\nPower Sepctrum Parameters:")
print("  kmin   [1/Mpc] = ", kmin)
print("  kmax   [1/Mpc] = ", kmax)


Power Sepctrum Parameters:
  kmin   [1/Mpc] =  0.047936899621426284
  kmax   [1/Mpc] =  24.543692606170257


In [ ]:
# Cosmological Parameters
cosmo_params = {
    'z_start': z_start,
    'Lbox_cMpc': c_Lbox_Mpc,
    'Lbox_cMpc/h': c_Lbox_Mpc*h,
    'dx_fin_kpc': dx_fin_kpc,
    'levelmin': levelmin,
    'levelmax': levelmax,
    'FlagAMR': FlagAMR,
    'FlagZoom': FlagZoom,
    'k0': k0,
    'As': As0, # trial number (before normalization)
    'ns': ns,
    'h': h,
    'bias8_z0_WMAP5': bias8_z0_WMAP5,
    'Omega_m': Omega_m,
    'Omega_b': Omega_b,
    'Omega_k': Omega_k,
    'w0': w0,
    'wa': wa,
    'kmax': kmax,
    'k_per_logint': k_per_logint
}

cosmo_params['sigma8_z0_WMAP5'] = 1/cosmo_params['bias8_z0_WMAP5']
cosmo_params['Omega_c'] = cosmo_params['Omega_m'] - cosmo_params['Omega_b']
cosmo_params['Omega_l'] = 1 - cosmo_params['Omega_m']

In [9]:
cosmo_params

{'z_start': 200,
 'Lbox_cMpc': 131.072,
 'Lbox_cMpc/h': 94.37183999999999,
 'dx_fin_kpc': 1,
 'levelmin': 10,
 'levelmax': 17,
 'FlagAMR': 1,
 'FlagZoom': 0,
 'k0': 0.05,
 'As': 2e-09,
 'ns': 0.96,
 'h': 0.72,
 'bias8_z0_WMAP5': 1.26,
 'Omega_m': 0.26,
 'Omega_b': 0.044,
 'Omega_k': 0.0,
 'w0': -1,
 'wa': 0,
 'kmax': 24.543692606170257,
 'k_per_logint': 0,
 'sigma8_z0_WMAP5': 0.7936507936507936,
 'Omega_c': 0.21600000000000003,
 'Omega_l': 0.74}

In [10]:
z = [z_start, 150, 100, 20, 10, 1, 0] # should contain z_start and 0
z.sort(reverse=True) # should be descending order

In [11]:
pars    = initialize_camb_params(cosmo_params, z=z) # initialize
pars    = adjust_As_at_z0(pars, cosmo_params) # normalization

Target     sigma8 = 0.7936507936507936 at z = 0.0
Initial    sigma8 = 0.7628693474928042, As = 2e-09 (trial)
Normalized sigma8 = 0.7936507936507937, diff=1.1102230246251565e-16, As = 2.1646544338903384e-09 (new)


In [12]:
cosmo_params['As'] = pars.InitPower.As # update the As value

In [13]:
results   = camb.get_results(pars)
transfers = results.get_matter_transfer_data()
T = transfers.transfer_data
T.shape # [var, k, z]

(13, 217, 7)

In [14]:
header = "\n".join([f"{key}: {value}" for key, value in cosmo_params.items()])
print(header)

z_start: 200
Lbox_cMpc: 131.072
Lbox_cMpc/h: 94.37183999999999
dx_fin_kpc: 1
levelmin: 10
levelmax: 17
FlagAMR: 1
FlagZoom: 0
k0: 0.05
As: 2.1646544338903384e-09
ns: 0.96
h: 0.72
bias8_z0_WMAP5: 1.26
Omega_m: 0.26
Omega_b: 0.044
Omega_k: 0.0
w0: -1
wa: 0
kmax: 24.543692606170257
k_per_logint: 0
sigma8_z0_WMAP5: 0.7936507936507936
Omega_c: 0.21600000000000003
Omega_l: 0.74


In [15]:
# iz = z.index(z_start)
iz = z.index(0)

In [16]:
plus_w0 = '+' if w0 >= 0 else ''
plus_wa = '+' if wa >= 0 else ''
fname = f"camb_transfer_z{z[iz]:03.0f}_cpl{plus_w0}{w0:.1f}{plus_wa}{wa:.1f}_lmin{levelmin:02d}.txt"
np.savetxt(f"./{fname}", T[:,:,iz].T, fmt="%.10e", header=header)
print(f"MUSIC input data is created: ./{fname}")

MUSIC input data is created: ./camb_transfer_z000_cpl-1.0+0.0_lmin10.txt


In [17]:
print("The number of k values = ", T.shape[1])

The number of k values =  217
